<img src="http://imgur.com/1ZcRyrc.png" style="float: left; margin: 20px; height: 55px">

-----


## Training an ANN on the Reuters newswire topic dataset with Keras

-----

In [1]:
'''Trains and evaluate a simple ANN
on the Reuters newswire topic classification task.
'''
import numpy as np
import matplotlib.pyplot as plt
import keras
from keras.datasets import reuters
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
#from keras.layers.noise import AlphaDropout
from keras.layers.noise import GaussianDropout
from keras.preprocessing.text import Tokenizer
import itertools
import pandas as pd





/home/ohalloranb/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


** Set up the training and test sets **

In [2]:
max_words = 1000
batch_size = 32
epochs = 5

print('Loading data...')
(x_train, y_train), (x_test, y_test) = reuters.load_data(num_words=max_words,
                                                         test_split=0.2)
print(len(x_train), 'train sequences')
print(len(x_test), 'test sequences')


Loading data...
2113536/2110848 [==============================] - 2s 1us/step
8982 train sequences
2246 test sequences


In [3]:
test = reuters.load_data(num_words=max_words)

** Vectorise the sets **

In [4]:
import numpy as np
num_classes = np.max(y_train) + 1
print(num_classes, 'classes')

print('Vectorizing sequence data...')
tokenizer = Tokenizer(num_words=max_words)
x_train = tokenizer.sequences_to_matrix(x_train, mode='binary')
x_test = tokenizer.sequences_to_matrix(x_test, mode='binary')
print('x_train shape:', x_train.shape)
print('x_test shape:', x_test.shape)

print('Convert class vector to binary class matrix '
      '(for use with categorical_crossentropy)')
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)
print('y_train shape:', y_train.shape)
print('y_test shape:', y_test.shape)


46 classes
Vectorizing sequence data...
x_train shape: (8982, 1000)
x_test shape: (2246, 1000)
Convert class vector to binary class matrix (for use with categorical_crossentropy)
y_train shape: (8982, 46)
y_test shape: (2246, 46)


** Train the model **

In [5]:
print('Building model...')
model = Sequential()
model.add(Dense(512, input_shape=(max_words,)))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1)

Building model...
Train on 8083 samples, validate on 899 samples
Epoch 1/5
8083/8083 [==============================] - 1s 171us/step - loss: 1.4288 - acc: 0.6790 - val_loss: 1.0907 - val_acc: 0.7642
Epoch 2/5
8083/8083 [==============================] - 1s 152us/step - loss: 0.7884 - acc: 0.8178 - val_loss: 0.9397 - val_acc: 0.7864
Epoch 3/5
8083/8083 [==============================] - 1s 153us/step - loss: 0.5506 - acc: 0.8663 - val_loss: 0.8959 - val_acc: 0.7998
Epoch 4/5
8083/8083 [==============================] - 1s 153us/step - loss: 0.4162 - acc: 0.8987 - val_loss: 0.8775 - val_acc: 0.8053
Epoch 5/5
8083/8083 [==============================] - 1s 152us/step - loss: 0.3248 - acc: 0.9169 - val_loss: 0.9098 - val_acc: 0.8009


** Evaluate the model **

In [6]:
score = model.evaluate(x_test, y_test,
                       batch_size=batch_size, verbose=1)


2246/2246 [==============================] - 0s 33us/step


In [7]:
print('Test accuracy:', score[1])



Test accuracy: 0.7929652715939448


** Make class predictions, returns the probability by class **

In [8]:
classes = model.predict(x_test, batch_size=1)


In [9]:
classes

array([[4.32701154e-06, 3.38561267e-06, 7.02602492e-08, ...,
        1.99522006e-07, 3.11058095e-08, 6.04913453e-08],
       [6.06576726e-03, 3.63966972e-02, 1.06007062e-01, ...,
        1.39281910e-05, 1.69916457e-05, 1.22127458e-04],
       [1.15170529e-04, 9.93882775e-01, 5.38385248e-05, ...,
        5.75645072e-06, 1.05636282e-05, 6.62861748e-06],
       ...,
       [2.69963020e-05, 3.61952378e-04, 3.79072771e-05, ...,
        7.27510815e-06, 3.44066939e-06, 1.32327878e-05],
       [8.66306829e-04, 9.67240147e-03, 1.24871079e-03, ...,
        6.56119009e-05, 3.97442745e-05, 2.49422355e-05],
       [3.88807064e-04, 7.07652152e-01, 3.88066936e-03, ...,
        1.17377094e-04, 3.22795640e-05, 6.52925082e-05]], dtype=float32)

** Extact the top 6 class probabilities and return **

In [10]:
N=6
threshold = 2.5e-1

indices = np.argsort(classes, axis=1)[:, -N:]
indices = np.fliplr(indices)
indices_list = indices.tolist()
chain = list(itertools.chain(*indices))
probs = classes[0, chain]
probs = probs[probs>threshold]


In [11]:
res_df = pd.DataFrame({'Pred_Class_1':indices[:,0],'Pred_Class_2':indices[:,1],'Pred_Class_3':indices[:,2],
                         'Pred_Class_2':indices[:,1], 'Pred_Class_3':indices[:,2], 'Pred_Class_4':indices[:,3], 'Pred_Class_5':indices[:,4], 'Pred_Class_6':indices[:,5]})

In [12]:
res_df.head()

,Pred_Class_1,Pred_Class_2,Pred_Class_3,Pred_Class_4,Pred_Class_5,Pred_Class_6
0,3,4,13,20,8,16
1,10,24,2,1,28,4
2,1,3,28,13,5,16
3,16,4,17,3,41,33
4,13,3,16,1,2,23
